# The Problem of Loanwords: detection and remedies

# Detect Transliterated Greek
### In this notebook you will:
1. ETL (Extract-Transform-Load) data
    1. Extract data using a Corpus Reader object
    1. Transform data using a reusable, composable Scikit-Learn Pipeline object
    1. Load the data into a data matrix and a classifier
1. Train several classifiers and select the best algorithm for the data
1. Use GridSearch to tune hyperparameters to achieve the best algorithm performance
1. Use the classifier to predict whether or not a word is transliterated Greek
1. Examine some unseen, untrained data to discover how well the classifier generalizes to unseen data
1. Save the classifier for reuse
1. Record the trained classifier model's provenance
## Why? 
### If you desire to make a high quality word embedding, you will probably want to filter out Greek words which may appear transliterated into Latin. Often Latin authors will quote transliterated Greek words (sometimes transliterated Greek words are valid Latin words, but most often they can pollute a computational view of the language). 
#### The data sets are: the works of Vergil, Eutropius, and Plato's Apologia transliterated into Latin. We will use our classifier to examine the corpus of Cicero to detect the use of transliterated Greek words, and we'll assess the classifier's effectiveness on the entire latin library corpus

In [2]:
%load_ext autoreload
%autoreload 2
%doctest_mode on
%matplotlib inline
import warnings
warnings.simplefilter('ignore') # quiet warnings for presentation purposes only

Exception reporting mode: Plain
Doctest mode is: ON


### Imports

In [12]:
import datetime
import glob
import json
import logging
import multiprocessing
import os
from copy import deepcopy

from tqdm import tqdm
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.extmath import density
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.externals.joblib import dump, load
import sklearn
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from cltk.prosody.latin.scansion_constants import ScansionConstants
from cltk.prosody.latin.string_utils import remove_punctuation_dict
from cltk.corpus.readers import get_corpus_reader
from cltk.utils.featurization import word_to_features
from cltk.utils.file_operations import md5
from cltk.utils.matrix_corpus_fun import (
    distinct_words,
    separate_camel_cases,
    drop_empty_lists,
    drop_non_lower,
    drop_arabic_numeric,
    drop_all_caps,
    drop_empty_strings,
    jv_transform,
    splice_hyphens,
    accept_editorial,
    profile_chars,
    demacronize,
    drop_enclitics,
    drop_fringe_punctuation,
    divide_separate_words,
    drop_all_punctuation)
from building_language_model.aeoe_replacer import aeoe_transform

### Turn on logging, primarily so that library methods may report warnings

In [13]:
LOG = logging.getLogger('make_model')
LOG.addHandler(logging.NullHandler())
logging.basicConfig(level=logging.INFO)

### Define a corpus reader and remap the _fileids to the text directory containing the corpus of Livy

In [18]:
reader = get_corpus_reader('latin_text_latin_library', language='latin')
ALL_FILE_IDS = list(reader.fileids())
good_files = [ file for file in ALL_FILE_IDS
              if 'vergil' in file or
              'eutropius' in file]

LOG.info('available good files %s', len(good_files))
reader._fileids = good_files
good_files
                   

INFO:make_model:available good files 36


['eutropius/eutropius1.txt', 'eutropius/eutropius10.txt', 'eutropius/eutropius2.txt', 'eutropius/eutropius3.txt', 'eutropius/eutropius4.txt', 'eutropius/eutropius5.txt', 'eutropius/eutropius6.txt', 'eutropius/eutropius7.txt', 'eutropius/eutropius8.txt', 'eutropius/eutropius9.txt', 'vergil/aen1.txt', 'vergil/aen10.txt', 'vergil/aen11.txt', 'vergil/aen12.txt', 'vergil/aen2.txt', 'vergil/aen3.txt', 'vergil/aen4.txt', 'vergil/aen5.txt', 'vergil/aen6.txt', 'vergil/aen7.txt', 'vergil/aen8.txt', 'vergil/aen9.txt', 'vergil/ec1.txt', 'vergil/ec10.txt', 'vergil/ec2.txt', 'vergil/ec3.txt', 'vergil/ec4.txt', 'vergil/ec5.txt', 'vergil/ec6.txt', 'vergil/ec7.txt', 'vergil/ec8.txt', 'vergil/ec9.txt', 'vergil/geo1.txt', 'vergil/geo2.txt', 'vergil/geo3.txt', 'vergil/geo4.txt']

### Define a custom Scikit-learn Pipeline, and call the CorpusReader `sents()` method to process the texts
#### The functions used in the pipelines are doctest documented in the `corpus_cleaning` module
#### The functions used and their order was developed iteratively by running the pipelines on actual data and carefully inspecting the results.

In [20]:
process_text_model = Pipeline([
    ('separate_camel_cases', FunctionTransformer(separate_camel_cases, validate=False)),
    ('splice_hyphens', FunctionTransformer(splice_hyphens, validate=False)),
    ('jv_transform', FunctionTransformer(jv_transform, validate=False)),
    ('aeoe_transform', FunctionTransformer(aeoe_transform, validate=False)),
    ('accept_editorial', FunctionTransformer(accept_editorial, validate=False)),
    ('drop_enclitics', FunctionTransformer(drop_enclitics, validate=False)),
    ('drop_fringe_punctuation', FunctionTransformer(drop_fringe_punctuation, validate=False)),
    ('drop_all_punctuation', FunctionTransformer(drop_all_punctuation, validate=False)),
    ('drop_non_lower', FunctionTransformer(drop_non_lower, validate=False)),
    ('drop_arabic_numeric', FunctionTransformer(drop_arabic_numeric, validate=False)),
    ('drop_all_caps', FunctionTransformer(drop_all_caps, validate=False)),
    ('divide_separate_words', FunctionTransformer(divide_separate_words, validate=False)),
    ('drop_empty_lists', FunctionTransformer(drop_empty_lists, validate=False)),
    ('drop_empty_strings', FunctionTransformer(drop_empty_strings, validate=False))])

X = process_text_model.fit_transform(tqdm(list(reader.sents())))

100%|██████████| 6580/6580 [00:00<00:00, 38023.29it/s] | 3897/6580 [00:00<00:00, 38966.14it/s]


### Analyze the resulting matrix, by profiling the character occurences and go back and adjust the pipeline as necessary, and turn the output into a distinct set of words

In [22]:
char_count = profile_chars(X)
print('Character distribution profile, total chars:', sum(char_count.values()))
print(char_count)
distinct_good_latin = distinct_words(X)
print('Number of distinct words in Eutropius/Vergil sample', len(distinct_good_latin))

Character distribution profile, total chars: 590072
Counter({'e': 69370, 'i': 59205, 'u': 56178, 'a': 55052, 't': 47491, 's': 44518, 'r': 39859, 'n': 35666, 'm': 30375, 'o': 29541, 'c': 22855, 'l': 19290, 'p': 15088, 'd': 14245, 'q': 9674, 'b': 8559, 'g': 7453, 'f': 5935, 'h': 4656, 'x': 2791, 'A': 1653, 'y': 1152, 'T': 1088, 'P': 987, 'C': 915, 'I': 898, 'M': 690, 'S': 685, 'R': 543, 'D': 530, 'L': 511, 'H': 508, 'N': 383, 'E': 348, 'U': 290, 'G': 267, 'O': 222, 'B': 198, 'F': 159, 'Q': 148, 'z': 39, 'Z': 22, 'K': 13, 'X': 12, 'k': 10})
Number of distinct words in Eutropius/Vergil sample 22899


### Load the transliterated Greek examples, profile the raw character counts, and go back and tune the pipeline if necessary

In [23]:
transliterated_greek_file = 'greek.transliterated.plato.apologia.txt'   
greek_transliterated_X = []

with open(transliterated_greek_file, 'rt') as reader:
    greek_transliterated_X.append(reader.read().split())

print('Unprocessed character distribution profile of transliterated Greek',
      profile_chars(greek_transliterated_X))

Unprocessed character distribution profile of transliterated Greek Counter({'i': 4480, 'a': 4459, 'e': 4386, 'o': 4298, 't': 3851, 'n': 3647, 's': 2523, 'h': 2279, 'u': 2200, 'ō': 1544, 'p': 1518, 'k': 1503, 'ē': 1447, 'm': 1435, 'l': 1292, 'r': 1181, 'd': 1092, 'g': 836, ',': 805, '.': 263, "'": 205, 'b': 170, ':': 138, 'x': 126, 'z': 110, ';': 82, 'A': 74, '—': 53, '’': 43, 'M': 36, '‘': 32, 'S': 24, 'D': 11, 'K': 11, 'H': 11, 'P': 10, 'T': 9, 'L': 7, 'E': 3, 'O': 3, 'G': 1, 'Ē': 1, 'N': 1, 'R': 1})


### Process the transliterated Greek examples with another custom Scikit-learn Pipeline, analyze character profiles for tuning, create a set distinct words, with and without macrons

In [24]:
process_greek_transliterated_model = Pipeline([
    ('splice_hyphens', FunctionTransformer(splice_hyphens, validate=False)),
    ('divide_separate_words', FunctionTransformer(divide_separate_words, validate=False)),
    ('drop_fringe_punctuation', FunctionTransformer(drop_fringe_punctuation, validate=False)),
    ('drop_all_punctuation', FunctionTransformer(drop_all_punctuation, validate=False)),
    ('drop_empty_strings', FunctionTransformer(drop_empty_strings, validate=False)),
    ('drop_empty_lists', FunctionTransformer(drop_empty_lists, validate=False))])

greek_X = process_greek_transliterated_model.fit_transform(greek_transliterated_X)
print('Character distribution profile of transliterated Greek', profile_chars(greek_X))
distinct_transliterated_greek_examples = distinct_words(greek_X)
print('distinct_transliterated_greek_examples', len(distinct_transliterated_greek_examples))
demarcronized_greek_X = demacronize(greek_X)
distinct_transliterated_greek_examples = distinct_transliterated_greek_examples | distinct_words(
    demarcronized_greek_X)
print('distinct_transliterated_greek_examples with & w/out macrons',
      len(distinct_transliterated_greek_examples))

Character distribution profile of transliterated Greek Counter({'i': 4480, 'a': 4459, 'e': 4386, 'o': 4298, 't': 3851, 'n': 3647, 's': 2523, 'h': 2279, 'u': 2200, 'ō': 1544, 'p': 1518, 'k': 1503, 'ē': 1447, 'm': 1435, 'l': 1292, 'r': 1181, 'd': 1092, 'g': 836, 'b': 170, 'x': 126, 'z': 110, 'A': 74, 'M': 36, 'S': 24, 'D': 11, 'K': 11, 'H': 11, 'P': 10, 'T': 9, 'L': 7, 'E': 3, 'O': 3, 'G': 1, 'Ē': 1, 'N': 1, 'R': 1})
distinct_transliterated_greek_examples 2406
distinct_transliterated_greek_examples with & w/out macrons 3356


### Remove any words from the transliterated Greek words which have also appear in the Latin corpus

In [27]:
shared_words = distinct_transliterated_greek_examples & distinct_livy
print('Shared_words:', shared_words)
only_greek_transliterated = distinct_transliterated_greek_examples - shared_words
print('Number of distinct transliterated Greek without matching words in the Latin corpus',
      len(only_greek_transliterated))
# compare with Livy, should probably add some, e.g.: eis

Shared_words: {'lego', 'Orphei', 'dia', 'ito', 'arista', 'heroas', 'hoste', 'ei', 'par', 'esto', 'tonde', 'deo', 'para', 'pater', 'Rhadamanthus', 'polo', 'ara', 'de', 'te', 'Troia', 'meta', 'o', 'iste', 'se', 'tot', 'ergo', 'hostis', 'en', 'duo', 'ne', 'hora', 'ex', 'no', 'porro', 'aedes', 'asto', 'pateras', 'pro', 'omen', 'Di', 'horas', 'an', 'dei', 'este', 'lege', 'di', 'ego', 'hos', 'Salamina', 'Minos', 'ea', 'me', 'nux'}
Number of distinct transliterated Greek without matching words in the Latin corpus 3303


### Create a simple data matrix of the single words, transliterated Greek examples followed by the Latin words

In [28]:
X = [list(only_greek_transliterated) + list(distinct_good_latin)]
len(X[0])

26202

### Before we transform our data matrix into a data feature matrix, let's check on the max word lengths

In [30]:
def get_max_word_len(col):
    max_word = 0
    for word in col:
        max_word = max(len(word), max_word)
    return max_word

print('Max word length in Eutropius/Vergil sample', get_max_word_len(distinct_livy))
print('Max word length in transliterated Greek sample', get_max_word_len(only_greek_transliterated))
max_len = max(20, max(get_max_word_len(only_greek_transliterated), get_max_word_len(distinct_livy)))
# for testing the feature making function below set 
# max_len=17

Max word length in Eutropius/Vergil sample 17
Max word length in transliterated Greek sample 18


In [31]:
# This method is included in corpus_cleaning.py but we include it here for easy reference
def word_to_features(word, max_word_length=20):
    """
    Convert a single word into an array of numbers based on character ordinals, with padding
    :param word: a single word
    :param max_word_length: the maximum word length for the feature array
    :return: a list of integers padded to the max word length

    >>> word_to_features('far', 20)
    [116, 114, 97, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]
    """
    if len(word) > max_word_length:
        LOG.warning('Excessive word length {} for {}, truncating to {}'.format(len(word), word,
                                                                               max_word_length))
        word = word[:max_word_length]
    word = list(word)
    word.reverse() #: encourage aligning on word endings if possible
    return [ord(c) for c in "".join(word).ljust(max_word_length, ' ')]


In [32]:
all_y = np.array([1] * len(only_greek_transliterated) + [0] * len(distinct_livy), dtype=float)
print('y shape', all_y.shape)
# We use a label encoder to automatically capture the range of values for provenance
label_encoder = LabelEncoder()
label_encoder.fit(all_y)
all_words = list(only_greek_transliterated) + list(distinct_livy)
all_X = np.array([word_to_features(word, max_len) for word in all_words])
print('X shape', all_X.shape)
num_samples = all_y.shape[0] # to be used later by model provenance
num_features = all_X.shape[1] # to be used later by model provenance

y shape (26202,)
X shape (26202, 20)


### Train a DummyClassifier to show the baseline which we must improve above

In [33]:
dummy = DummyClassifier(strategy='stratified', random_state=0)
features_train, features_test, target_train, target_test = train_test_split(all_X, all_y,
                                                                            random_state=0)
dummy.fit(features_train, target_train)
dummy_score = dummy.score(features_test, target_test)
print('Dummy classifier: {}'.format(dummy_score))

Dummy classifier: 0.7742329415356434


### Train and classify the data using several classifiers, printing out the cross validation score results

In [35]:
classifiers = [
    MultinomialNB,
    GaussianNB,
    KNeighborsClassifier,
    BaggingClassifier,
    ExtraTreesClassifier,
    RandomForestClassifier
]
for cls in tqdm(classifiers):
    scores = cross_val_score(cls(), all_X, all_y,
                             scoring='accuracy',
                             n_jobs=multiprocessing.cpu_count(),
                             cv=5)
    print('{} {} {}'.format(str(cls), scores.mean(), scores))

  0%|          | 0/6 [00:00<?, ?it/s] 33%|███▎      | 2/6 [00:00<00:00, 14.13it/s]

<class 'sklearn.naive_bayes.MultinomialNB'> 0.7741388912184812 [0.778859   0.76874642 0.78172105 0.7730916  0.76827639]
<class 'sklearn.naive_bayes.GaussianNB'> 0.8919550476176541 [0.89105133 0.89047892 0.89601221 0.88244275 0.89979004]


 50%|█████     | 3/6 [00:01<00:01,  2.47it/s]

<class 'sklearn.neighbors.classification.KNeighborsClassifier'> 0.9197773125067078 [0.92081664 0.92043503 0.91566495 0.91946565 0.92250429]


 67%|██████▋   | 4/6 [00:02<00:01,  1.97it/s]

<class 'sklearn.ensemble.bagging.BaggingClassifier'> 0.956949732157588 [0.95401641 0.95745087 0.96126693 0.95629771 0.95571674]


 83%|████████▎ | 5/6 [00:02<00:00,  2.34it/s]

<class 'sklearn.ensemble.forest.ExtraTreesClassifier'> 0.9439356081672641 [0.94123259 0.94428544 0.94504865 0.94561069 0.94350067]


100%|██████████| 6/6 [00:02<00:00,  2.59it/s]

<class 'sklearn.ensemble.forest.RandomForestClassifier'> 0.9520266214577967 [0.95000954 0.95153597 0.95306239 0.95343511 0.95209009]


#### Note: Training on 10% of Livy yielded a dummy classifier of 50%, and a BaggingClassifier with 90%. By using more data, the dummy heuristic goes up, but so does the ceiling of the classifier's ultimate score.
### Run Grid Search to optimize one of the best classifiers

In [36]:
grids = GridSearchCV(cv=5, error_score='raise',
                     estimator=BaggingClassifier(
                         base_estimator=DecisionTreeClassifier(class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features=None,
                                                               max_leaf_nodes=None,
                                                               min_samples_leaf=1,
                                                               min_samples_split=2,
                                                               min_weight_fraction_leaf=0.0,
                                                               presort=False,
                                                               random_state=1)), n_jobs=-1,
                     param_grid={
                         # Here's how to test variations on the super class estimator
                         #     'base_estimator__criterion': ['gini'], # 'entropy'],
                         'n_estimators': [10, 100, 200],
                         # Here are some other ranges tested
                     #  150, 250, 300], # 50, 70, 100, 150, 200, 250, 300],
                         'max_features': [0.2, 0.4, 0.7, 1.0],
                         'max_samples': [0.5, 1.0]  # 0.6, 0.8,
                     })
grids.fit(all_X, all_y)
print('Best score: %s', grids.best_score_)
print('Best params %s', grids.best_params_)

# Best score: %s 0.9589360709286048
# Best params %s {'base_estimator__criterion': 'gini', 'max_features': 0.4, 'max_samples': 1.0, 'n_estimators': 200}


Best score: %s 0.9667964277536066
Best params %s {'max_features': 0.7, 'max_samples': 1.0, 'n_estimators': 100}


### Using the best parameters from GridSearch, build the optimal classifier

In [37]:
# First let's copy the parameters for the provenance file
mdl_params = deepcopy(grids.best_params_)
# Let's also remove the base_estimator parameters, since they aren't honored by the constructor, unlike GridSearch
if 'base_estimator__criterion' in mdl_params:
    del mdl_params['base_estimator__criterion']
classifier = BaggingClassifier(**mdl_params)
classifier.fit(all_X, all_y)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=0.7, max_samples=1.0,
         n_estimators=100, n_jobs=None, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

### Let's look at Cicero for transliterated Greek words 

In [42]:
greek_in_cicero = set()
reader = get_corpus_reader(corpus_name='latin_text_latin_library', language='latin')

cicero = [ file for file in reader.fileids()
         if 'cicero' in file ]
for file in tqdm(cicero):
    reader = get_corpus_reader(corpus_name='latin_text_latin_library', language='latin')
    reader._fileids = [file]
    unseen_X = process_text_model.fit_transform(list(reader.sents()))
    distinct_unseen = distinct_words(unseen_X)
    print('Checking: {} : {:,} words'.format(file[file.rfind('/') + 1:], len(distinct_unseen)))
    unseen_words = list(distinct_unseen)
    if unseen_words:
        arr = classifier.predict(
            np.array([word_to_features(word, max_len) for word in unseen_words]))
        marks = arr.tolist()
        found_greek = [unseen_words[idx]
                       for idx, point in enumerate(marks)
                       if point == 1]
        greek_in_cicero |= set(found_greek)
        print('found: {:,} transliterated Greek words: {}'.format(np.count_nonzero(arr), found_greek))
print('Number of Greek words not in training data: {:,}'.format(len(greek_in_cicero - only_greek_transliterated)))

  0%|          | 0/138 [00:00<?, ?it/s]  1%|          | 1/138 [00:00<02:10,  1.05it/s]

Checking: acad.txt : 2,057 words
found: 3 transliterated Greek words: ['eis', 'eita', 'philosophandi']


  1%|▏         | 2/138 [00:02<02:15,  1.00it/s]

Checking: adbrutum1.txt : 2,885 words
found: 4 transliterated Greek words: ['ep', 'coniunctiori', 'emphatikôtero', 'med']


  2%|▏         | 3/138 [00:02<02:02,  1.10it/s]

...

Checking: verres.2.3.txt : 6,563 words
found: 16 transliterated Greek words: ['Leontinos', 'praetermitti', 'existimaturi', 'substituisti', 'Ephesi', 'adulescentuli', 'ßiil', 'med', 'constituisti', 'querimonias', 'discrimen', 'administrasti', 'eis', 'reprehendisti', 'instrumento', 'eme']


 99%|█████████▉| 137/138 [02:33<00:01,  1.69s/it]

Checking: verres.2.4.txt : 5,515 words
found: 9 transliterated Greek words: ['comperendinato', 'Archagatho', 'peruenissentne', 'circumueniri', 'peripetasmatis', 'querimonias', 'peripetasmata', 'intellexisti', 'instrumento']


100%|██████████| 138/138 [02:35<00:00,  1.77s/it]

Checking: verres.2.5.txt : 5,915 words
found: 6 transliterated Greek words: ['conlocauisti', 'octaphoro', 'pronuntiasti', 'discrimen', 'eis', 'interrogasti']
Number of Greek words not in training data: 844


### Note: ML is not failsafe like set intersection; a classifier predicts based on features that are aggregates of the sample space; e.g. notice the classifier is marking `non` as transliterated Greek even when it's not mismatched in the samples.

In [43]:
print('non' in distinct_good_latin)
print('non' in distinct_transliterated_greek_examples)
# TODO find a better example

True
False


### ...With a larger training set, this minor inconsistency would probably disappear.
#### However, the `-on` ending is common noun and adjective accusative declension in Greek, and the Latin word `non` is an anomaly.
#### Also, 98.5% accuracy still means 1.5% inaccuracy so there is a cost for AI assessments.
### Save the classifier for future use using the sklearn joblib library

In [44]:

def rle(inarray):
        """Run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                  # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = np.array(ia[1:] != ia[:-1])     # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])
            
# cribbed from stack overflow, needs annotations, and it's bit overkill but many possible uses

In [47]:
word='agathon'
classifier.predict(
            np.array([word_to_features(word, max_len)]))

array([1.])

### Find which texts of livy have the least greek

In [56]:
 
latin_files= glob.glob(
    os.path.expanduser('~/cltk_data/latin/text/latin_text_latin_library//livy/*'))


In [65]:
corpus_files = {}  
for full_path in tqdm(latin_files) :
    filename = full_path[full_path.rfind('/') + 1:]
    reader = get_corpus_reader(corpus_name='latin_text_latin_library', language='latin')
    reader._fileids =    [full_path]
    unseen_X = process_text_model.fit_transform(list(reader.sents()))
    distinct_unseen = distinct_words(unseen_X)    
    unseen_words = list(distinct_unseen)    
    total_words =[word
                for sentence in unseen_X
                for word in sentence]    
    arr = classifier.predict(
            np.array([word_to_features(word, max_len) for word in total_words]))
    total_greek_words = np.count_nonzero(arr)
    marks = arr.tolist()
    if marks:
        found_greek = [total_words[idx]
                       for idx, point in enumerate(marks)
                       if point == 1]
        corpus_files [filename]=  ( len(total_words), total_greek_words)
        print('file: {} total words: {:,} with {:,} transliterated Greek words: {}'.format(
            filename, len(total_words), total_greek_words, found_greek ))        





  0%|          | 0/178 [00:00<?, ?it/s]



  1%|          | 1/178 [00:00<01:45,  1.67it/s]

file: liv.per67.txt total words: 171 with 0 transliterated Greek words: []






  1%|          | 2/178 [00:01<01:42,  1.71it/s]

file: liv.per128.txt total words: 50 with 0 transliterated Greek words: []






  2%|▏         | 3/178 [00:01<01:42,  1.71it/s]

...

file: liv.per42.txt total words: 107 with 0 transliterated Greek words: []






 99%|█████████▉| 177/178 [02:48<00:00,  1.58it/s]

file: liv.per125.txt total words: 69 with 0 transliterated Greek words: []






100%|██████████| 178/178 [02:49<00:00,  1.63it/s]





file: liv.per131.txt total words: 64 with 0 transliterated Greek words: []


In [66]:
print(len(corpus_files ))
rankings = [ (key, val[0], val[1], val[1]/val[0]) for key, val in corpus_files.items()]
rankings.sort(key=lambda x: x[3])
for rank in rankings:
    print (rank)

178
('liv.per67.txt', 171, 0, 0.0)
('liv.per128.txt', 50, 0, 0.0)
('liv.per73.txt', 127, 0, 0.0)
('liv.per114.txt', 92, 0, 0.0)
('liv.per100.txt', 68, 0, 0.0)
('liv.per98.txt', 116, 0, 0.0)
('liv.per99.txt', 103, 0, 0.0)
('liv.per101.txt', 77, 0, 0.0)
('liv.per115.txt', 77, 0, 0.0)
('liv.per72.txt', 70, 0, 0.0)
('liv.per129.txt', 80, 0, 0.0)
('liv.per66.txt', 41, 0, 0.0)
('liv.per136-7.txt', 4, 0, 0.0)
('liv.per64.txt', 111, 0, 0.0)
('liv.per103.txt', 140, 0, 0.0)
('liv.per58.txt', 198, 0, 0.0)
('liv.per117.txt', 125, 0, 0.0)
('liv.per116.txt', 182, 0, 0.0)
('liv.per59.txt', 339, 0, 0.0)
('liv.per102.txt', 97, 0, 0.0)
('liv.per65.txt', 72, 0, 0.0)
('liv.per71.txt', 83, 0, 0.0)
('liv.per106.txt', 111, 0, 0.0)
('liv.per49.txt', 706, 0, 0.0)
('liv.per112.txt', 129, 0, 0.0)
('liv.per75.txt', 96, 0, 0.0)
('liv.per61.txt', 150, 0, 0.0)
('liv.per60.txt', 206, 0, 0.0)
('liv.per113.txt', 129, 0, 0.0)
('liv.per48.txt', 511, 0, 0.0)
('liv.per107.txt', 133, 0, 0.0)
('liv.per111.txt', 106, 0, 0.0)


In [ ]:
model_output_file = 'is_transliterated_greek.mdl.{}.joblib'.format(sklearn.__version__)
dump(classifier, model_output_file)

### Reconstitute the classifier for use at runtime

In [ ]:
classifier = load(model_output_file)

### Some demo examples

In [ ]:
classifier.predict(
    np.array([word_to_features(word, max_len) for word in ['quid', 'est', 'veritas']]))

In [ ]:
classifier.predict(
    np.array([word_to_features(word, max_len) for word in ['ou', 'eis', 'panta', 'ton']]))

### Save model provenance 
#### Provenance -  a record of how, when, and with what the classifier was made with.
#### Important for allowing others to use the classifier in the future without rebuilding from scratch, or determining how it may be recreated for better or different performance

In [ ]:
data_files = {}
idx = 0
for idx, file in enumerate(files):
    data_files[idx] = {"filename": file[file.rfind("/") + 1:],
                       "md5": md5(file)
                       }
data_files[idx + 1] = {
    "filename": transliterated_greek_file,
    "md5": md5(transliterated_greek_file)
}

provenance_file = '{}.prov.json'.format(model_output_file)

params = {
    "provenance_data": provenance_file,
    "date_created": str(datetime.datetime.now()),
    "model_parameters": mdl_params,
    "max_word_length": max_len,
    "num_samples": num_samples,
    "num_features": num_features,
    "library_version": sklearn.__version__,
    "classifier_class": "{}".format(str(classifier.__class__ )),
    "classifier_best_score": grids.best_score_,
    "data_files": data_files,
    "model_output_file": model_output_file,
    "model_output_md5": md5(model_output_file),
    "labels": label_encoder.classes_.tolist(),
    # manually added information
    "comment": "Transliterated Greek Classifier",
    "code_generated_by": "detect_transliterated_greek.ipynb",
    "feature_encoding_fun": "word_to_features",
    "author": "Todd Cook"
}

with open(provenance_file, 'wt') as writer:
    json.dump(params, writer, indent=2)
    print('Wrote provenance file: {}'.format(provenance_file))


# Appendix

### How about that model provenance file? It should be readable, here it is:

In [ ]:
print(params)

# That's all folks!